## Transformations used to convert original dataset to match SOPHY

In [30]:
import os
import pandas as pd
import numpy as np
import sqlite3
import cartopy.crs as ccrs
import geopandas as gpd
from geopandas import GeoDataFrame
from pandas import DataFrame, Series, Index

In [19]:
con: sqlite3.Connection = sqlite3.connect(":memory:")
with open('schema.sql', 'r') as sql_file:
    con.executescript(sql_file.read())
con.commit()

In [34]:
sophy_xlsx_output: str = '../data/out/sophy.xlsx'
zones_shapefile: str = '../data/out/shapefiles/zones/so_zones.shp'
lter1_file: str = "../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Cruise_forEDI.csv"
lter2_file:str = "../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Station_forEDI.csv"
worms_dir: str = '../data/in/worms/'
phytobase_file: str = '../data/in/datasets/mod/phytobase.csv'
phytobase_worms_file = '../data/in/worms/phytobase_worms.csv'
joywarren_file: str = '../data/in/datasets/modified/joy_warren.csv'
joywarren_worms_file: str = '../data/in/worms/joy_warren_worms.csv'
joywarren_chemtax_file: str = '../data/in/datasets/modified/joy_warren_chemtax.csv'
joywarren_microscopy_file: str = '../data/in/datasets/modified/joy_warren_microscopy.csv'
units_csv: str = '../data/in/units.csv'

# worms output -> sql col name. Also include columns from the original data that are needed but don't need renaming
# Ex: "class" -> "class" means col name is correct but class column is needed for calculations and/or used in database
worms_sql: dict = {"AphiaID": "aphia_id", "scientificname": "name", "authority": "authority",
                   "superkingdom": "superkingdom",
                   "kingdom": "kingdom", "phylum": "phylum", "subphylum": "subphylum", "superclass": "superclass",
                   "class": "class", "subclass": "subclass", "superorder": "superorder", "order": "orders",
                   "suborder": "suborder", "infraorder": "infraorder", "superfamily": "superfamily",
                   "family": "family", "genus": "genus", "species": "species", "modified": "modified"}

def get_table_cols(table: str) -> tuple:
    """List of columns in SQL table"""
    data = con.execute(f"pragma table_info({table})").fetchall()
    if data: return list(zip(*data))[1]
    else: raise ValueError(f"Table {table} does not exist in the database")

def get_taxonomy() -> DataFrame:
    """Writes stored WoRMS queries to database"""
    result = None
    for file in os.listdir(worms_dir):
        if file.endswith('.csv'):
            new = pd.read_csv(worms_dir + file, encoding='utf-8').drop_duplicates(subset=['AphiaID']).rename(
                columns=worms_sql)
            result = pd.concat([result, new], ignore_index=True)
    result = result.drop_duplicates(subset=['aphia_id'])
    return result

1) Palmer LTER dataset
- Link to source and info

In [21]:
lter1 = pd.read_csv(lter1_file)
lter2 = pd.read_csv(lter2_file)

lter_sql: dict = {"DatetimeGMT": "timestamp", "Latitude": "latitude", "Longitude": "longitude",
                  "Depth": "depth", "Temperature": "temperature", "Salinity": "salinity", "Density": "density",
                  "Chlorophyll": "chl_a", "Fluorescence": "fluorescence", "Phaeopigment": "phaeopigments",
                  "PrimaryProduction": "primary_prod", "studyName": "cruise", "PAR": "par",
                  "Prasinophytes": "chemtax_prasinophytes", "Cryptophytes": "chemtax_cryptophytes",
                  "MixedFlagellates": "chemtax_mixed_flagellates", "Diatoms": "chemtax_diatoms",
                  "Haptophytes": "chemtax_haptophytes", "NO3": "nitrate", "NO2": "nitrite",
                  "DIC1": "diss_inorg_carbon", "DOC": "diss_org_carbon", "POC": "part_org_carbon",
                  "SiO4": "silicate", "N": "tot_nitrogen", "PO4": "phosphate", "Notes1": "notes"}

lter1 = lter1.rename(columns=lter_sql)
lter2 = lter2.rename(columns=lter_sql)
lter: DataFrame = pd.concat([lter1, lter2])
lter = lter.dropna(subset=['timestamp', 'longitude', 'latitude'])
lter = lter[lter['longitude'].between(-180, 180)]
lter = lter[lter['latitude'] <= -30]
# Group chemtax into three main categories
lter['percent_phaeo'] = lter['chemtax_haptophytes']
lter['percent_diatom'] = lter['chemtax_diatoms']
lter['percent_other'] = lter['chemtax_prasinophytes'] + lter['chemtax_mixed_flagellates'] + lter['chemtax_cryptophytes']

data_gdf = GeoDataFrame(lter, geometry=gpd.points_from_xy(lter['longitude'], lter['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
lter = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
# Labels data by Southern Ocean sectors (bins) and their longitude range
# Ross Sea sector overlaps with the start and end of range: [-180, 180] so it is defined with two split ranges
sectors_series: Series = pd.cut(lter['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
lter = lter.assign(sector=sectors_series)

extra: Index = lter.columns.difference(get_table_cols("sample"))
lter["extra_json"] = lter[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
lter = lter.drop(extra, axis=1)
lter

,cruise,timestamp,latitude,longitude,depth,temperature,salinity,par,chl_a,chemtax_prasinophytes,...,phosphate,silicate,nitrite,nitrate,percent_phaeo,percent_diatom,percent_other,front_zone,sector,extra_json
0,PD91-09,1991-11-07 00:36:00,-64.83333,-64.05167,2.14998,-1.7618,33.791,NaN,0.397000,0.139939,...,NaN,NaN,NaN,NaN,0.000000,0.000603,0.999397,SIZ,BA,"{""Alloxanthin"":0.0,""BetaCarotene"":0.00219978,""..."
1,PD91-09,1991-11-07 00:36:00,-64.83333,-64.05167,10.48500,-1.7445,33.798,NaN,0.298000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""BottleNumber"":11.0,""Event"":1.0,""FilterCode"":..."
2,PD91-09,1991-11-07 00:36:00,-64.83333,-64.05167,20.44900,-1.6974,33.820,NaN,0.265000,0.116048,...,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,SIZ,BA,"{""Alloxanthin"":0.0,""BetaCarotene"":0.00393294,""..."
3,PD91-09,1991-11-07 00:36:00,-64.83333,-64.05167,30.60000,-1.6188,33.846,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""Event"":1.0,""GridLine"":619.0,""GridRegion"":""NC..."
4,PD91-09,1991-11-07 00:36:00,-64.83333,-64.05167,40.54660,-1.4717,33.879,NaN,0.284000,0.002492,...,NaN,NaN,NaN,NaN,0.073917,0.388492,0.537591,SIZ,BA,"{""Alloxanthin"":0.0,""BetaCarotene"":0.00564388,""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19341,PAL2021,2021-05-04 00:00:00,-64.81500,-64.04050,20.00000,NaN,NaN,NaN,0.533521,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""Event"":54.0,""FilterCode"":5.0,""JulianDay"":124..."
19342,PAL2021,2021-05-04 00:00:00,-64.81500,-64.04050,35.00000,NaN,NaN,NaN,0.488506,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""Event"":54.0,""FilterCode"":5.0,""JulianDay"":124..."
19343,PAL2021,2021-05-04 00:00:00,-64.81500,-64.04050,50.00000,NaN,NaN,NaN,0.430152,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""Event"":54.0,""FilterCode"":5.0,""JulianDay"":124..."
19344,PAL2021,2021-05-04 00:00:00,-64.81500,-64.04050,65.00000,NaN,NaN,NaN,0.355125,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIZ,BA,"{""Event"":54.0,""FilterCode"":5.0,""JulianDay"":124..."


In [22]:
lter.to_sql(name='sample', con=con, if_exists='append', index=False)

49225

2) Joy-Warren 2019 dataset
- Link to source and info

In [25]:
"""Writes Joy-Warren 2019 dataset to database"""
joyw: DataFrame = pd.read_csv(joywarren_file, encoding='utf-8')
# group by depth and station: station1([1.7, 1.8, 2.1][9.7. 9.8, 10.2]...), station2...
joyw = joyw.groupby([joyw["depth"].pct_change().abs().gt(0.15).cumsum(), "station"]).mean(numeric_only=True)
joyw = joyw.reset_index(level=0, drop=True).reset_index(level=0).sort_values(by='depth')
joyw.sort_values(by=['station', 'depth'])
# --------------------------------------------------
jwchemtax: DataFrame = pd.read_csv(joywarren_chemtax_file)
jwchemtax = jwchemtax.dropna().sort_values(by='depth')
# --------------------------------------------------
# Join main sample data with chemtax by matching station and depth
joyw: DataFrame = pd.merge_asof(jwchemtax, joyw, by='station', on='depth', direction='nearest',
                                  tolerance=2).sort_values(by='station')

joyw['timestamp'] = pd.to_datetime(joyw['date'], format='%Y%m%d', errors='coerce').dropna().drop(
    columns=['date', 'time'])
joyw['source_name'] = 'joyw'
joyw['percent_phaeo'] = joyw['chemtax_haptophytes']
joyw['percent_diatom'] = joyw['chemtax_diatoms']
joyw['percent_other'] = joyw['chemtax_chlorophytes'] + joyw['chemtax_mixed_flagellates'] + joyw['chemtax_cryptophytes']
# ----------------------------------------------------
# Set id field for sample data so foreign keys for microscopy can be matched
joyw = joyw.reset_index(drop=True)
max_id: int = pd.read_sql("select max(id) from sample", con=con)['max(id)'][0] + 1
joyw['id'] = np.arange(max_id, max_id + len(joyw))
jwmkey: DataFrame = pd.concat([joyw['id'], joyw['station'], joyw['depth']], axis=1).sort_values(by='depth')
# Group extra columns into JSON
extra: Index = joyw.columns.difference(get_table_cols("sample"))
joyw["extra_json"] = joyw[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
joyw = joyw.drop(columns=extra, axis=1)

microscopy: DataFrame = pd.read_csv(joywarren_microscopy_file, encoding='utf-8').dropna()
replace: tuple = ('centric', 'pennate', 'unknown diatom', 'dinoflagellate', 'ciliate', 'silicoflagellate')
# are_taxa = rows that have species name we can get taxonomy for (ex: Phaeocystis)
are_taxa: Series = ~microscopy['taxa'].isin(replace)
taxa: DataFrame = pd.read_csv(joywarren_worms_file, encoding='utf-8').rename(worms_sql)
taxa.index = microscopy[are_taxa].index
# ----------------------------------
microscopy['aphia_id'] = taxa['AphiaID']
microscopy = microscopy.sort_values(by='depth')
# Join microscopy data with matching id in sample table (by depth and station)
microscopy = pd.merge_asof(microscopy, jwmkey, by='station', on='depth', direction='nearest', tolerance=1)
microscopy = microscopy.rename({'id': 'sample_id', 'taxa': 'name', 'group': 'groups'},
                               axis="columns")
# Remove extra columns
microscopy = microscopy[microscopy.columns.intersection(get_table_cols("microscopy"))]

data_gdf = GeoDataFrame(joyw, geometry=gpd.points_from_xy(joyw['longitude'], joyw['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labels for zones and sectors
joyw = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right'])).drop_duplicates(subset=['id'])
sectors_series: Series = pd.cut(joyw['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
joyw = joyw.assign(sector=sectors_series)
joyw

,depth,chemtax_chlorophytes,chemtax_mixed_flagellates,chemtax_cryptophytes,chemtax_haptophytes,chemtax_diatoms,latitude,longitude,temperature,salinity,...,silicate,timestamp,source_name,percent_phaeo,percent_diatom,percent_other,id,extra_json,front_zone,sector
0,9.8,0.001490,0.000002,0.032421,0.478822,0.487265,-62.7033,-69.6903,-0.677414,33.799114,...,22.600,2014-10-31,joyw,0.478822,0.487265,0.033913,49226,"{""CTDFLUOR_UP"":0.9224714286,""CTDOXY2_UP"":8.049...",ASZ,BA
1,3.5,0.048396,0.023013,0.034700,0.423520,0.470371,-62.7033,-69.6903,-0.677300,33.798900,...,NaN,2014-10-31,joyw,0.423520,0.470371,0.106109,49227,"{""CTDFLUOR_UP"":0.90615,""CTDOXY2_UP"":8.04509,""C...",ASZ,BA
2,76.3,0.103945,0.046908,0.021846,0.397119,0.430182,-62.7033,-69.6903,-0.681500,33.799200,...,NaN,2014-10-31,joyw,0.397119,0.430182,0.172699,49228,"{""CTDFLUOR_UP"":0.8833,""CTDOXY2_UP"":8.05498,""CT...",ASZ,BA
3,24.7,0.001383,0.000830,0.028901,0.350910,0.617978,-62.7033,-69.6903,-0.677767,33.799067,...,NaN,2014-10-31,joyw,0.350910,0.617978,0.031113,49229,"{""CTDFLUOR_UP"":0.9033333333,""CTDOXY2_UP"":8.044...",ASZ,BA
4,100.1,0.039702,0.015726,0.028563,0.357956,0.558053,-62.7033,-69.6903,-0.681550,33.800650,...,NaN,2014-10-31,joyw,0.357956,0.558053,0.083991,49230,"{""CTDFLUOR_UP"":0.86,""CTDOXY2_UP"":8.03891,""CTDO...",ASZ,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,9.9,0.072893,0.006168,0.029439,0.459095,0.432405,-65.8500,-71.3753,-1.709362,33.805200,...,NaN,2014-11-21,joyw,0.459095,0.432405,0.108500,49472,"{""CTDFLUOR_UP"":1.5700625,""CTDOXY2_UP"":8.010118...",SIZ,BA
247,99.2,0.000000,0.000000,0.000000,0.474952,0.525048,-65.8500,-71.3753,1.409000,34.363800,...,65.970,2014-11-21,joyw,0.474952,0.525048,0.000000,49473,"{""CTDFLUOR_UP"":0.0202,""CTDOXY2_UP"":4.76686,""CT...",SIZ,BA
248,1.3,0.085356,0.004651,0.028932,0.469119,0.411942,-65.8500,-71.3753,-1.687700,33.802200,...,44.440,2014-11-21,joyw,0.469119,0.411942,0.118939,49474,"{""CTDFLUOR_UP"":1.3039,""CTDOXY2_UP"":8.02862,""CT...",SIZ,BA
249,48.8,0.065835,0.000000,0.000000,0.512621,0.421544,-65.8500,-71.3753,-1.443133,33.945317,...,51.825,2014-11-21,joyw,0.512621,0.421544,0.065835,49475,"{""CTDFLUOR_UP"":0.5041833333,""CTDOXY2_UP"":7.346...",SIZ,BA


In [26]:
joyw.to_sql(name='sample', con=con, if_exists='append', index=False)
microscopy.to_sql(name='microscopy', con=con, if_exists='append', index=False)

137

3) Phytobase
- Link to source and info

In [27]:
phybase: DataFrame = pd.read_csv(phytobase_file)
phybase['timestamp'] = pd.to_datetime(phybase['timestamp'], errors='coerce')
# get full taxonomy of microscopy data as dataframe
taxa: DataFrame = pd.read_csv(phytobase_worms_file)[['original', 'AphiaID']]
# join on sample and taxonomy (by aphia_id), only keep cols in the occurrence table (filter out order, genus, etc)
phybase = pd.merge(phybase, taxa, left_on='scientificname', right_on='original')
phybase = phybase.rename(columns=worms_sql).filter(get_table_cols("occurrence"))

data_gdf = GeoDataFrame(phybase, geometry=gpd.points_from_xy(phybase['longitude'], phybase['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
phybase = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
sectors_series: Series = pd.cut(phybase['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
phybase = phybase.assign(sector=sectors_series)
phybase

,source_name,aphia_id,name,latitude,longitude,timestamp,depth,front_zone,sector
0,phytobase,620590.0,Coccopterum labyrinthus,-41.0001,-74.4499,2002-04-12,0.0,SAZ,BA
1,phytobase,620590.0,Coccopterum labyrinthus,-41.0001,-74.4499,2002-04-12,1.0,SAZ,BA
2,phytobase,620590.0,Coccopterum labyrinthus,-41.0001,-74.4499,2002-04-12,10.0,SAZ,BA
3,phytobase,620590.0,Coccopterum labyrinthus,-41.0001,-74.4499,2002-04-12,11.0,SAZ,BA
4,phytobase,620590.0,Coccopterum labyrinthus,-41.0001,-74.4499,2002-04-12,12.0,SAZ,BA
...,...,...,...,...,...,...,...,...,...
38278,phytobase,NaN,Picoeukaryotes,-39.9900,95.0100,1995-09-29,51.0,STZ,WPO
38279,phytobase,NaN,Picoeukaryotes,-43.0000,95.0100,1995-09-28,0.0,STZ,WPO
38280,phytobase,NaN,Picoeukaryotes,-43.0000,95.0100,1995-09-28,48.0,STZ,WPO
38281,phytobase,NaN,Picoeukaryotes,-31.7500,95.0000,1995-10-02,0.0,STZ,WPO


In [28]:
phybase.to_sql(name='occurrence', con=con, if_exists='append', index=False)

38438

# SQLite Database to Excel
Converts all tables to Excel workbook with multiple sheets (tables)

In [35]:
writer = pd.ExcelWriter(sophy_xlsx_output, engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
pd.read_sql('select * from sample', con=con).to_excel(writer, sheet_name='sample')
pd.read_sql('select * from microscopy', con=con).to_excel(writer, sheet_name='microscopy')
pd.read_sql('select * from occurrence', con=con).to_excel(writer, sheet_name='occurrence')
get_taxonomy().to_excel(writer, sheet_name='taxonomy')
pd.read_csv(units_csv).to_excel(writer, sheet_name='units')
# Close the Pandas Excel writer and output the Excel file.
writer.close()

4) Alderkamp dataset
- Link to source and info

5) Garibotti dataset
- Link to source and info